# Machine Translation

In [1]:
!nvidia-smi
!lscpu

Wed Jul  7 05:02:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install hazm==0.7.0
!pip install seqeval==1.2.2
!pip install sentencepiece==0.1.96
!pip install transformers==4.7.0
!pip install clean-text[gpl]==0.4.0
!pip install sacrebleu==1.5.1

     |████████████████████████████████| 317kB 28.1MB/s 
     |████████████████████████████████| 235kB 42.7MB/s 
     |████████████████████████████████| 1.4MB 36.8MB/s 
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154153 sha256=f91ce4516ea00d4b5d8fa6b2183dd04eeb16014f0697936300394841a89fdcc8
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
  Created wheel for nltk: filename=nltk-3.3-cp37-none-any.whl size=1394490 sha256=49d2332ebafe997dcddf8482ef8c9b6b3729b535b0fd0853d55e1ee07b6b8716
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built libwapiti nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 51kB 6.2MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16184 sha256=4ae91293300f4d

In [3]:
!pip install PyDrive
import os
import IPython.display as ipd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# Import required packages
import os
import gc
import re
import hazm
import time
import json
import sacrebleu
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import transformers
from transformers import AutoConfig, AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import MT5Config, MT5ForConditionalGeneration, MT5Tokenizer

from cleantext import clean

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print()
print('numpy', np.__version__)
print('pandas', pd.__version__)
print('transformers', transformers.__version__)
print('torch', torch.__version__)
print()

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



numpy 1.19.5
pandas 1.1.5
transformers 4.7.0
torch 1.9.0+cu102

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
class MachineTranslationDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Machine Translation. """

    def __init__(self, original_text, translated_text, tokenizer, max_length):
        self.original_text = original_text
        self.translated_text = translated_text
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.original_text)

    def __getitem__(self, item):
        encoding = self.tokenizer(
            self.original_text[item],
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        )
        inputs = {
            'original': self.original_text[item],
            'translated': self.translated_text[item],
            'input_ids': encoding.input_ids.flatten(),
            'attention_mask': encoding.attention_mask.flatten()
        }
        return inputs


class MachineTranslation:
    def __init__(self, model_name, model_type):
        self.normalizer = hazm.Normalizer()
        self.model_name = model_name
        if model_type.lower() == "mt5":
            self.tokenizer = MT5Tokenizer.from_pretrained(model_name)
            self.model = MT5ForConditionalGeneration.from_pretrained(model_name)
            self.config = MT5Config.from_pretrained(self.model_name)
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name)
            self.config = AutoConfig.from_pretrained(self.model_name)
            self.id2label = self.config.id2label
            self.label2id = self.config.label2id

    def load_dataset_test_file(self, dataset_name, dataset_file, **kwargs):
        if dataset_name.lower() == "mizan":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            data = pd.read_csv(dataset_file, delimiter="\t", names=['original', 'translation'], header=None)
            original, translation = data['original'].values.tolist(), data['translation'].values.tolist()
            print(f'test part:\n #original: {len(original)}, #translation: {len(translation)}')
            return original, translation
        if dataset_name.lower() == "combined":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            data = pd.read_csv(dataset_file, delimiter="\t", names=['original', 'translation', 'source'], header=None)
            if 'source' in kwargs:
                data = data[data['source'] == kwargs['source']]
                data = data[['original', 'translation']]
            else:
                data = data[['original', 'translation']]
            original, translation = data['original'].values.tolist(), data['translation'].values.tolist()
            print(f'test part:\n #original: {len(original)}, #translation: {len(translation)}')
            return original, translation

    def load_dataset_file(self, dataset_name, dataset_file, **kwargs):
        if dataset_name.lower() == "quran" or dataset_name.lower() == "bible":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            original, translation = [], []
            with open(dataset_file, encoding="utf8") as infile:
                for line in infile:
                    parts = line.strip().split('\t')
                    original.append(parts[0])
                    translation.append(parts[1])
            print(f'all data:\n #original: {len(original)}, #translation: {len(translation)}')

            _, original_test, _, translation_test = train_test_split(original, translation, test_size=0.1,
                                                                     random_state=1)
            print(f'test part:\n #original: {len(original_test)}, #translation: {len(translation_test)}')
            return original, translation, original_test, translation_test

    def mt5_machine_translation_inference(self, input_text, device):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return

        tokenized_batch = self.tokenizer(
            input_text,
            padding=True,
            return_tensors="pt"
        )

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()

        input_ids = tokenized_batch.input_ids.to(device)
        attention_mask = tokenized_batch.attention_mask.to(device)
        outputs = self.model.generate(input_ids=input_ids,
                                      attention_mask=attention_mask)
        predictions = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return predictions

    def mt5_evaluation(self, input_text, translated_text, device, max_length, split_reference=None, batch_size=4):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return
        if len(input_text) != len(translated_text):
            print('length of inputs and its translations is not equal!!')
            return

        dataset = MachineTranslationDataset(original_text=input_text, translated_text=translated_text,
                                            tokenizer=self.tokenizer, max_length=max_length)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print(f'#original_text:{len(input_text)}, #translated_text:{len(translated_text)}')
        print("#batch:", len(data_loader))

        if split_reference is None:
            max_num_ref = 1
        else:
            max_num_ref = 0
            for ref in translated_text:
                max_num_ref = max(max_num_ref, len(ref.split(split_reference)))
        print("#maximum_translation_reference:", max_num_ref)

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()

        total_time = 0
        output_predictions = []
        golden_translations, predicted_translations = [[] for _ in range(max_num_ref)], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_input_ids = batch['input_ids']
            b_attention_mask = batch['attention_mask']

            # move tensors to GPU if CUDA is available
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model.generate(input_ids=b_input_ids, attention_mask=b_attention_mask)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')

            b_predictions = self.tokenizer.batch_decode(b_outputs, skip_special_tokens=True)

            for i in range(len(b_input_ids)):
                if split_reference is None:
                    sample_golden_translation = [batch['translated'][i]]
                else:
                    sample_golden_translation = batch['translated'][i].split(split_reference)
                sample_generated_translation = b_predictions[i]
                bleu_score = sacrebleu.corpus_bleu(sys_stream=[sample_generated_translation],
                                                   ref_streams=[[g] for g in sample_golden_translation]).score
                output_predictions.append((batch['original'][i], batch['translated'][i], b_predictions[i], bleu_score))

                for j in range(max_num_ref):
                    try:
                        golden_translations[j].append(sample_golden_translation[j])
                    except:
                        golden_translations[j].append('')
                predicted_translations.append(sample_generated_translation)

        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(input_text))

        # evaluate
        print("BLEU Score: {}".format(sacrebleu.corpus_bleu(
            sys_stream=predicted_translations, ref_streams=golden_translations).score))
        return output_predictions


In [6]:
model_name='persiannlp/mt5-small-parsinlu-translation_en_fa'
mt_model = MachineTranslation(model_name=model_name, model_type="mt5")
print(mt_model.config)


MT5Config {
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.7.0",
  "use_cache": true,
  "vocab_size": 250112
}



## Sample Inference

In [ ]:
input_list = [
  "Praise be to Allah, the Cherisher and Sustainer of the worlds;",
  "shrouds herself in white and walks penitentially disguised as brotherly love through factories and parliaments; offers help, but desires power;",
  "He thanked all fellow bloggers and organizations that showed support.",
  "Races are held between April and December at the Veliefendi Hippodrome near Bakerky, 15 km (9 miles) west of Istanbul.",
  "I want to pursue PhD in Computer Science about social network,what is the open problem in social networks?"
]
mt_model.mt5_machine_translation_inference(input_list, device)

['برای الله، یعنی چرنده و سوزان دنیا، تحسین کنید',
 'خودش را در سفید پوسته می کند و به صورت عشق برادرانه',
 'او از تمام بلاگرها و سازمان هایی که حمایتشان را نشان می داد',
 'در طول ماه آوریل و دسامبر در والی فیودورونا نزدیک بیکر',
 'من می خواهم در مورد شبکه اجتماعی تحقیقات علوم کامپیوتری را دن']

## Mizan Dataset


In [ ]:
!wget https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/mizan/mizan_test_en_fa.tsv
!ls

--2021-07-03 19:44:35--  https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/mizan/mizan_test_en_fa.tsv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2296459 (2.2M) [text/plain]
Saving to: ‘mizan_test_en_fa.tsv’

mizan_test_en_fa.ts 100%[===================>]   2.19M  --.-KB/s    in 0.07s   

2021-07-03 19:44:36 (32.6 MB/s) - ‘mizan_test_en_fa.tsv’ saved [2296459/2296459]

adc.json  mizan_test_en_fa.tsv	sample_data


In [ ]:
mizan_original_text, mizan_translated_text = mt_model.load_dataset_test_file(dataset_name="mizan", dataset_file="mizan_test_en_fa.tsv")
print(mizan_original_text[:5])
print(mizan_translated_text[:5])
print(len(mizan_original_text))
print(len(mizan_translated_text))

test part:
 #original: 10000, #translation: 10000
['a sure sign with him of mental agitation.', ';Cesare, you are worried about this affair,; she said at last.', ';I am very sorry you feel so despondent over it;', 'but I could decide only as seemed right to me.;', ';It is not the affair,; he answered, sullenly; ;I know nothing about it,']
['این نشان قاطعی از هیجان وی بود شروع به قدم زدن در اتاق کرد.', 'عاقبت گفت: سزار! تو از این موضوع نگران شده\u200cای؛', 'بسیار متأسفم که نسبت به آن احساس بی اعتمادی می\u200cکنی؛', 'اما من فقط بدان علت نتوانستم تصمیم بگیرم که آن کار به نظرم صحیح رسید.', 'مارتینی، با اخم، پاسخ داد: به خاطر آن موضوع نیست، من در آن باره چیزی نمی\u200cدانم؛']
10000
10000


In [ ]:
mt_model.mt5_machine_translation_inference(mizan_original_text[:5], device)

['نشانه ای از اضطراب ذهنی او.',
 ':کریر، تو نگران این قضیه هستی. سرانجام گفت:',
 'متاسفم که احساس غم و اندوه می کنی.',
 'اما من فقط می توانم تصمیم بگیرم.',
 ':این قضیه نیست، و با خشونت پاسخ داد: من چیزی']

In [ ]:
!nvidia-smi
!lscpu

Sat Jul  3 19:44:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    28W /  70W |   2276MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(mizan_original_text, mizan_translated_text, device, max_length=512, batch_size=128)

#original_text:10000, #translated_text:10000
#batch: 79
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 2.0573687799999902
inference time for step 1: 1.9872234450000121
inference time for step 2: 1.9957603619999986
inference time for step 3: 2.0018457820000037
inference time for step 4: 2.0140770460000113
inference time for step 5: 2.0117879350000294
inference time for step 6: 2.0281689779999965
inference time for step 7: 2.031034216000023
inference time for step 8: 2.026235602999975
inference time for step 9: 2.04356522300003
inference time for step 10: 2.0226792810000234
inference time for step 11: 2.028123029000028
inference time for step 12: 2.0310351170000445
inference time for step 13: 2.046099830000003
inference time for step 14: 2.042858272999979
inference time for step 15: 2.046773628999972
inference time for step 16: 2.0531158050000045
inference time for step 17: 2.0470818339999823
inference time for step 18: 2.0502682520000235
inf

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

a sure sign with him of mental agitation.	این نشان قاطعی از هیجان وی بود شروع به قدم زدن در اتاق کرد.	نشانه ای از اضطراب ذهنی او.	2.4906123264252495
;Cesare, you are worried about this affair,; she said at last.	عاقبت گفت: سزار! تو از این موضوع نگران شده‌ای؛	:کریر، تو نگران این قضیه هستی. سرانجام گفت:	9.425159511373677
;I am very sorry you feel so despondent over it;	بسیار متأسفم که نسبت به آن احساس بی اعتمادی می‌کنی؛	متاسفم که احساس غم و اندوه می کنی.	5.0735520042259505
but I could decide only as seemed right to me.;	اما من فقط بدان علت نتوانستم تصمیم بگیرم که آن کار به نظرم صحیح رسید.	اما من فقط می توانم تصمیم بگیرم.	9.95265213836697
;It is not the affair,; he answered, sullenly; ;I know nothing about it,	مارتینی، با اخم، پاسخ داد: به خاطر آن موضوع نیست، من در آن باره چیزی نمی‌دانم؛	:این قضیه نیست، و با خشونت پاسخ داد: من چیزی	10.27103870297191
and it probably is all right, once you have consented to go into it.	هنگامی که تو قبول می‌کنی در کاری شرکت جویی قطعا کاملا درست است.	و احتمال

In [ ]:
output_file_name = "translation_en-fa_mizan_test_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

## Quran Dataset

In [ ]:
!wget https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/quran/quran_en_fa.tsv
!ls

--2021-07-03 19:48:16--  https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/quran/quran_en_fa.tsv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16215125 (15M) [text/plain]
Saving to: ‘quran_en_fa.tsv’

quran_en_fa.tsv     100%[===================>]  15.46M  62.0MB/s    in 0.2s    

2021-07-03 19:48:18 (62.0 MB/s) - ‘quran_en_fa.tsv’ saved [16215125/16215125]

adc.json
mizan_test_en_fa.tsv
quran_en_fa.tsv
sample_data
translation_en-fa_mizan_test_persiannlp-mt5-small-parsinlu-translation_en_fa_outputs.txt


In [ ]:
quran_original_all, quran_translated_all, quran_original_test, quran_translated_test = mt_model.load_dataset_file(dataset_name="quran", dataset_file="quran_en_fa.tsv")
print(quran_original_all[0])
print(quran_translated_all[0])
print(quran_original_test[0])
print(quran_translated_test[0])

all data:
 #original: 6236, #translation: 6236
test part:
 #original: 624, #translation: 624
In the name of Allah, Most Gracious, Most Merciful.
به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خداى بخشاینده مهربان‌///به نام خداوند هستى بخش مهربان///به نام خداوند رحمتگر مهربان///به نام خداوند بخشنده‌ى مهربان.///به نام خداوند بخشنده مهربان///به نام خداوند بخشنده مهربان‌///به نام خداوند بخشنده بخشایشگر///بنام خداوند بخشاینده مهربان‌///به نام خدای رحمتگر بر آفریدگان، رحمتگر بر ویژگان‌
Nay, but man doth transgress all bounds,
این چنین نیست [که انسان سپاس گزار باشد] مسلما انسان سرکشی می‌کند.///این چنین نیست [که انسان سپاس گزار باشد] مسلما انسان سرکشی می‌کند.///حقا که آدمى نافرمانى مى‌کند،///چنین نیست [که مى‌پندارند]، بى‌شک آدمى طغیان مى‌کند///حقا که انسان سرکشى مى‌کند،///این چنین نیست [که انسان سپاس‌گزار باشد]، بى‌گمان آدمى سرکشى مى‌کند.///راستی که انسان سرکش و مغرور می‌شود.///چنین نیست، بی‌گمان انسان سر به طغیان برآورد

In [ ]:
mt_model.mt5_machine_translation_inference(quran_original_test[:5], device)

['نه، اما مرد تمام محدوده ها را رها می کند.',
 'مصری ها گفتند: "اگر شما ثابت کنید که دروغ گفتید چه',
 'پس هر دو به او بروید و بگویید: لطفا ما پیام رسان',
 'در مورد کسانی که ایمان را رد می کنند و مخالفت می کنند،',
 'دعا می کنم: «اوه خدای من! استخوان های من ضعیف']

### test set

In [ ]:
!nvidia-smi
!lscpu

Sat Jul  3 19:48:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    33W /  70W |   2286MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(quran_original_test, quran_translated_test, device, max_length=512, split_reference='///', batch_size=128)

#original_text:624, #translated_text:624
#batch: 5
#maximum_translation_reference: 11
Start to evaluate test data ...
inference time for step 0: 2.2231887470000515
inference time for step 1: 2.2001512209999987
inference time for step 2: 2.213750578000031
inference time for step 3: 2.2286363279999932
inference time for step 4: 2.012438551999992
total inference time: 10.878165426000066
total inference time / #samples: 0.017432957413461646
BLEU Score: 4.461483003323956


In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

Nay, but man doth transgress all bounds,	این چنین نیست [که انسان سپاس گزار باشد] مسلما انسان سرکشی می‌کند.///این چنین نیست [که انسان سپاس گزار باشد] مسلما انسان سرکشی می‌کند.///حقا که آدمى نافرمانى مى‌کند،///چنین نیست [که مى‌پندارند]، بى‌شک آدمى طغیان مى‌کند///حقا که انسان سرکشى مى‌کند،///این چنین نیست [که انسان سپاس‌گزار باشد]، بى‌گمان آدمى سرکشى مى‌کند.///راستی که انسان سرکش و مغرور می‌شود.///چنین نیست، بی‌گمان انسان سر به طغیان برآورد///چنین نیست (که شما می‌پندارید) به یقین انسان طغیان می‌کند،///نه چنین است همانا انسان سرکشی کند///چنان نیست (که انسان سرنهد). همواره انسان بی‌گمان سرکشی می‌کند،	نه، اما مرد تمام محدوده ها را رها می کند.	4.456882760699063
(The Egyptians) said: "What then shall be the penalty of this, if ye are (proved) to have lied?"	گماشتگان گفتند: اگر شما دروغگو باشید [و سارق در میان شما یافت شود] کیفرش چیست؟///گماشتگان گفتند: اگر شما دروغگو باشید [و سارق در میان شما یافت شود] کیفرش چیست؟///گفتند: اگر دروغ گفته باشید جزاى دزد چیست؟///گفتند: پس اگر دروغ گفته باشید کیفر

In [ ]:
output_file_name = "translation_en-fa_quran_test_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### all dataset

In [ ]:
!nvidia-smi
!lscpu

Sat Jul  3 19:48:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    34W /  70W |   5486MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(quran_original_all, quran_translated_all, device, max_length=512, split_reference='///', batch_size=128)

#original_text:6236, #translated_text:6236
#batch: 49
#maximum_translation_reference: 11
Start to evaluate test data ...
inference time for step 0: 2.2551495839999802
inference time for step 1: 2.269310485999995
inference time for step 2: 2.2715728389999867
inference time for step 3: 2.2799759520000293
inference time for step 4: 2.2804758249999963
inference time for step 5: 2.2895573900000272
inference time for step 6: 2.302630957999952
inference time for step 7: 2.3036515680000207
inference time for step 8: 2.2812920249999706
inference time for step 9: 2.264177621999977
inference time for step 10: 2.2670238519999657
inference time for step 11: 2.252805628000033
inference time for step 12: 2.2383720810000796
inference time for step 13: 2.230874713999924
inference time for step 14: 2.2094854039999063
inference time for step 15: 2.2167861170000833
inference time for step 16: 2.1952597240000387
inference time for step 17: 2.207870011000068
inference time for step 18: 2.1857896139999866
in

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the name of Allah, Most Gracious, Most Merciful.	به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خداى بخشاینده مهربان‌///به نام خداوند هستى بخش مهربان///به نام خداوند رحمتگر مهربان///به نام خداوند بخشنده‌ى مهربان.///به نام خداوند بخشنده مهربان///به نام خداوند بخشنده مهربان‌///به نام خداوند بخشنده بخشایشگر///بنام خداوند بخشاینده مهربان‌///به نام خدای رحمتگر بر آفریدگان، رحمتگر بر ویژگان‌	به نام الله، گرامی ترین و ممنون ترین.	11.339582221952005
Praise be to Allah, the Cherisher and Sustainer of the worlds;	همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///ستایش خدا را که پروردگار جهانیان است.///ستایش خداى را که پروردگار جهانیان است///ستایش خدایى را که پروردگار جهانیان،///سپاس و ستایش مخصوص خداوندى است که پروردگار جهانیان است.///ستایش خدای را که پروردگار جهانیان است.///سپاس خداوند را که پروردگار جهانیان است‌///ستایش مخصوص خداوندی است که پروردگار جهانیان است.///سپ

In [ ]:
output_file_name = "translation_en-fa_quran_all_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

## Bible Dataset

In [ ]:
!wget https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/bible/bible_en_fa.tsv
!ls

--2021-07-03 19:51:15--  https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/bible/bible_en_fa.tsv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10023337 (9.6M) [text/plain]
Saving to: ‘bible_en_fa.tsv’

bible_en_fa.tsv     100%[===================>]   9.56M  --.-KB/s    in 0.1s    

2021-07-03 19:51:16 (77.8 MB/s) - ‘bible_en_fa.tsv’ saved [10023337/10023337]

adc.json
bible_en_fa.tsv
mizan_test_en_fa.tsv
quran_en_fa.tsv
sample_data
translation_en-fa_mizan_test_persiannlp-mt5-small-parsinlu-translation_en_fa_outputs.txt
translation_en-fa_quran_all_persiannlp-mt5-small-parsinlu-translation_en_fa_outputs.txt
translation_en-fa_quran_test_persiannlp-mt5-small-parsinlu-translation_en_fa_outputs.txt


In [ ]:
bible_original_all, bible_translated_all, bible_original_test, bible_translated_test = mt_model.load_dataset_file(dataset_name="bible", dataset_file="bible_en_fa.tsv")
print(bible_original_all[0])
print(bible_translated_all[0])
print(bible_original_test[0])
print(bible_translated_test[0])

all data:
 #original: 31020, #translation: 31020
test part:
 #original: 3102, #translation: 3102
In the beginning God created the heaven and the earth.
در ابتدا، خدا آسمانها و زمین را آفرید.
And of the sons of Elizaphan; Shimri, and Jeiel: and of the sons of Asaph; Zechariah, and Mattaniah:
و چون قربانی های سوختنی تمام شد، پادشاه و جمیع حاضرین با وی رکوع کرده، سجده نمودند.


In [ ]:
mt_model.mt5_machine_translation_inference(bible_original_test[:5], device)

['و از پسران الیتیم، شیمری، جیلی: و پسران آصف،',
 'تمام ستون های اطراف دربار با نقره پوشیده می شوند. قل',
 'و موسی به آنان گفت: اگر بچه های خر و بچه های',
 'من به تو شیر و نه به گوشت غذا دادم: چون تا حالا',
 'و این اتفاق در پایان هفت روز گذشته بود که کلمه خداوند']

### test set

In [ ]:
!nvidia-smi
!lscpu

Sat Jul  3 19:51:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    36W /  70W |   2284MiB / 15109MiB |     25%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(bible_original_test, bible_translated_test, device, max_length=512, batch_size=128)

#original_text:3102, #translated_text:3102
#batch: 25
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 2.142086281000047
inference time for step 1: 2.1597508730000072
inference time for step 2: 2.1719656740000346
inference time for step 3: 2.186872753999978
inference time for step 4: 2.202379948999919
inference time for step 5: 2.2081501710000566
inference time for step 6: 2.242231543999992
inference time for step 7: 2.2705412489999617
inference time for step 8: 2.2889610669999456
inference time for step 9: 2.319296958999985
inference time for step 10: 2.3202385540000705
inference time for step 11: 2.337560714999995
inference time for step 12: 2.353729659999999
inference time for step 13: 2.327080987000045
inference time for step 14: 2.316024887000026
inference time for step 15: 2.2919783730000063
inference time for step 16: 2.282416155999954
inference time for step 17: 2.2596845729999586
inference time for step 18: 2.2379310159999477
inferen

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

And of the sons of Elizaphan; Shimri, and Jeiel: and of the sons of Asaph; Zechariah, and Mattaniah:	و چون قربانی های سوختنی تمام شد، پادشاه و جمیع حاضرین با وی رکوع کرده، سجده نمودند.	و از پسران الیتیم، شیمری، جیلی: و پسران آصف،	2.242173863464727
All the pillars round about the court shall be filleted with silver; their hooks shall be of silver, and their sockets of brass.	«و تو بنی‌اسرائیل را امر فرما که روغن زیتون مصفی و کوبیده شده برای روشنایی نزد توبیاورند تا چراغها دائم روشن شود.در خیمه اجتماع، بیرون پرده‌ای که در برابر شهادت است، هارون و پسرانش از شام تا صبح، به حضورخداوند آن را درست کنند. و این برای بنی‌اسرائیل نسلا بعد نسل فریضه ابدی باشد.	تمام ستون های اطراف دربار با نقره پوشیده می شوند. قل	0.07327492496430335
And Moses said unto them, If the children of Gad and the children of Reuben will pass with you over Jordan, every man armed to battle, before the LORD, and the land shall be subdued before you; then ye shall give them the land of Gilead for a possession:	وبنی جاد، دیبون

In [ ]:
output_file_name = "translation_en-fa_bible_test_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### all dataset

In [ ]:
!nvidia-smi
!lscpu

Sat Jul  3 19:52:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    33W /  70W |   5614MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(bible_original_all, bible_translated_all, device, max_length=512, batch_size=128)

#original_text:31020, #translated_text:31020
#batch: 243
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 2.19287535400008
inference time for step 1: 2.1696985789999417
inference time for step 2: 2.1666340250000076
inference time for step 3: 2.1710755540000264
inference time for step 4: 2.1649891320000734
inference time for step 5: 2.178352356000005
inference time for step 6: 2.167382115999999
inference time for step 7: 2.1648139669999864
inference time for step 8: 2.181807913000057
inference time for step 9: 2.177247373
inference time for step 10: 2.1986017159999847
inference time for step 11: 2.1986563220000335
inference time for step 12: 2.2062050019999333
inference time for step 13: 2.2098379789999854
inference time for step 14: 2.2219484769999553
inference time for step 15: 2.2172698430000537
inference time for step 16: 2.2212884120000354
inference time for step 17: 2.2344903329999397
inference time for step 18: 2.235866951000048
inferen

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the beginning God created the heaven and the earth.	در ابتدا، خدا آسمانها و زمین را آفرید.	در آغاز خدا بهشت و زمین را خلق کرده است.	16.59038701421971
And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.	وزمین تهی و بایر بود و تاریکی بر روی لجه. و روح خدا سطح آبها را فرو گرفت.	و زمین بدون شکل و خالی بود و تاریکی بر صورت عمیق	12.045972307298896
And God said, Let there be light: and there was light.	و خدا گفت: «روشنایی بشود.» و روشنایی شد.	و خداوند گفت: بگذار نور باشد: و نور هم وجود داشت.	7.141816289329644
And God saw the light, that it was good: and God divided the light from the darkness.	و خدا روشنایی را دید که نیکوست و خداروشنایی را از تاریکی جدا ساخت.	و خداوند نور را دید که خوب بود: و خداوند نور را از تاریکی	17.678748653651848
And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.	و خداروشنایی را روز نامید و تاریکی را شب نامید. وشام

In [ ]:
output_file_name = "translation_en-fa_bible_all_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

## translation_combined

In [7]:
!mkdir translation_combined_en_fa
!wget https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/translation_combined_en_fa/test.tsv -O translation_combined_en_fa/test.tsv
!ls translation_combined_en_fa

--2021-07-07 05:07:05--  https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/translation_combined_en_fa/test.tsv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29348716 (28M) [text/plain]
Saving to: ‘translation_combined_en_fa/test.tsv’

translation_combine 100%[===================>]  27.99M   152MB/s    in 0.2s    

2021-07-07 05:07:06 (152 MB/s) - ‘translation_combined_en_fa/test.tsv’ saved [29348716/29348716]

test.tsv


### All combined test set

In [8]:
combined_original_test, combined_translated_test = mt_model.load_dataset_test_file(dataset_name="combined", dataset_file="translation_combined_en_fa/test.tsv")
print(combined_original_test[0])
print(combined_translated_test[0])

test part:
 #original: 48123, #translation: 48123
In the beginning God created the heaven and the earth.
در ابتدا، خدا آسمانها و زمین را آفرید.


In [ ]:
!nvidia-smi
!lscpu

Sat Jul  3 20:02:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    33W /  70W |   5616MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(combined_original_test, combined_translated_test, device, max_length=512, batch_size=128)

#original_text:48123, #translated_text:48123
#batch: 376
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 2.190566616000069
inference time for step 1: 2.1614112170000226
inference time for step 2: 2.164823646000059
inference time for step 3: 2.190304504999858
inference time for step 4: 2.193522196999993
inference time for step 5: 2.2564316610000787
inference time for step 6: 2.2376951809999355
inference time for step 7: 2.2628556110000773
inference time for step 8: 2.2711285469999893
inference time for step 9: 2.2799434399998972
inference time for step 10: 2.2824164950000068
inference time for step 11: 2.285841623000124
inference time for step 12: 2.2730373260001215
inference time for step 13: 2.2508913340000163
inference time for step 14: 2.232790595000097
inference time for step 15: 2.2158227189997888
inference time for step 16: 2.2120749340001566
inference time for step 17: 2.2097907989998475
inference time for step 18: 2.196760433999998
i

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the beginning God created the heaven and the earth.	در ابتدا، خدا آسمانها و زمین را آفرید.	در آغاز خدا بهشت و زمین را خلق کرده است.	16.59038701421971
And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.	وزمین تهی و بایر بود و تاریکی بر روی لجه. و روح خدا سطح آبها را فرو گرفت.	و زمین بدون شکل و خالی بود و تاریکی بر صورت عمیق	12.045972307298896
And God said, Let there be light: and there was light.	و خدا گفت: «روشنایی بشود.» و روشنایی شد.	و خداوند گفت: بگذار نور باشد: و نور هم وجود داشت.	7.141816289329644
And God saw the light, that it was good: and God divided the light from the darkness.	و خدا روشنایی را دید که نیکوست و خداروشنایی را از تاریکی جدا ساخت.	و خداوند نور را دید که خوب بود: و خداوند نور را از تاریکی	17.678748653651848
And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.	و خداروشنایی را روز نامید و تاریکی را شب نامید. وشام

In [ ]:
output_file_name = "translation_en-fa_combined_all_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

In [9]:
!nvidia-smi
!lscpu

Wed Jul  7 05:07:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
evaluation_output = mt_model.mt5_evaluation(combined_original_test, combined_translated_test, device, max_length=512, split_reference='///', batch_size=128)

#original_text:48123, #translated_text:48123
#batch: 376
#maximum_translation_reference: 11
Start to evaluate test data ...
inference time for step 0: 2.267933219999975
inference time for step 1: 1.9995390130000033
inference time for step 2: 2.004232607000006
inference time for step 3: 2.014306777999991
inference time for step 4: 2.0233515630000056
inference time for step 5: 2.0319993459999637
inference time for step 6: 2.04098437600004
inference time for step 7: 2.042378601999985
inference time for step 8: 2.0542621120000035
inference time for step 9: 2.0373160099999836
inference time for step 10: 2.047176062999995
inference time for step 11: 2.064646989000039
inference time for step 12: 2.071521581000013
inference time for step 13: 2.0759691970000063
inference time for step 14: 2.08828562299999
inference time for step 15: 2.0996573319999925
inference time for step 16: 2.114520142999993
inference time for step 17: 2.1278404650000198
inference time for step 18: 2.1388069970000174
infer

In [11]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the beginning God created the heaven and the earth.	در ابتدا، خدا آسمانها و زمین را آفرید.	در آغاز خدا بهشت و زمین را خلق کرده است.	16.59038701421971
And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.	وزمین تهی و بایر بود و تاریکی بر روی لجه. و روح خدا سطح آبها را فرو گرفت.	و زمین بدون شکل و خالی بود و تاریکی بر صورت عمیق	12.045972307298896
And God said, Let there be light: and there was light.	و خدا گفت: «روشنایی بشود.» و روشنایی شد.	و خداوند گفت: بگذار نور باشد: و نور هم وجود داشت.	7.141816289329644
And God saw the light, that it was good: and God divided the light from the darkness.	و خدا روشنایی را دید که نیکوست و خداروشنایی را از تاریکی جدا ساخت.	و خداوند نور را دید که خوب بود: و خداوند نور را از تاریکی	17.678748653651848
And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.	و خداروشنایی را روز نامید و تاریکی را شب نامید. وشام

In [12]:
output_file_name = "translation_en-fa_combined_all_split-refs_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Bible subset

In [ ]:
bible_original_test, bible_translated_test = mt_model.load_dataset_test_file(dataset_name="combined", dataset_file="translation_combined_en_fa/test.tsv", source='bible_en_fa')
print(bible_original_test[0])
print(bible_translated_test[0])

test part:
 #original: 31020, #translation: 31020
In the beginning God created the heaven and the earth.
در ابتدا، خدا آسمانها و زمین را آفرید.


In [ ]:
!nvidia-smi
!lscpu

Sat Jul  3 20:17:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    32W /  70W |   5710MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(bible_original_test, bible_translated_test, device, max_length=512, batch_size=128)

#original_text:31020, #translated_text:31020
#batch: 243
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 2.162456218999978
inference time for step 1: 2.146930181999778
inference time for step 2: 2.159878409999692
inference time for step 3: 2.1851291499997387
inference time for step 4: 2.189863351999975
inference time for step 5: 2.2137015349999274
inference time for step 6: 2.238207486000192
inference time for step 7: 2.257159586999933
inference time for step 8: 2.2867005699999936
inference time for step 9: 2.3058320820000517
inference time for step 10: 2.3172854109998298
inference time for step 11: 2.339548066000134
inference time for step 12: 2.3640679559998716
inference time for step 13: 2.3245494369998596
inference time for step 14: 2.315590548000273
inference time for step 15: 2.290583720000086
inference time for step 16: 2.2734346570000525
inference time for step 17: 2.2512049130000378
inference time for step 18: 2.2434314259999155
inf

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the beginning God created the heaven and the earth.	در ابتدا، خدا آسمانها و زمین را آفرید.	در آغاز خدا بهشت و زمین را خلق کرده است.	16.59038701421971
And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.	وزمین تهی و بایر بود و تاریکی بر روی لجه. و روح خدا سطح آبها را فرو گرفت.	و زمین بدون شکل و خالی بود و تاریکی بر صورت عمیق	12.045972307298896
And God said, Let there be light: and there was light.	و خدا گفت: «روشنایی بشود.» و روشنایی شد.	و خداوند گفت: بگذار نور باشد: و نور هم وجود داشت.	7.141816289329644
And God saw the light, that it was good: and God divided the light from the darkness.	و خدا روشنایی را دید که نیکوست و خداروشنایی را از تاریکی جدا ساخت.	و خداوند نور را دید که خوب بود: و خداوند نور را از تاریکی	17.678748653651848
And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.	و خداروشنایی را روز نامید و تاریکی را شب نامید. وشام

In [ ]:
output_file_name = "translation_en-fa_combined_bible_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Mizan subset

In [ ]:
mizan_original_test, mizan_translated_test = mt_model.load_dataset_test_file(dataset_name="combined", dataset_file="translation_combined_en_fa/test.tsv", source='mizan_test_en_fa')
print(mizan_original_test[0])
print(mizan_translated_test[0])

test part:
 #original: 10000, #translation: 10000
a sure sign with him of mental agitation.
این نشان قاطعی از هیجان وی بود شروع به قدم زدن در اتاق کرد.


In [ ]:
!nvidia-smi
!lscpu

Sat Jul  3 20:27:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    33W /  70W |   5616MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(mizan_original_test, mizan_translated_test, device, max_length=512, batch_size=128)

#original_text:10000, #translated_text:10000
#batch: 79
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 2.176038309999967
inference time for step 1: 2.148170736999873
inference time for step 2: 2.1614454380001007
inference time for step 3: 2.1850534370000787
inference time for step 4: 2.1900885589998325
inference time for step 5: 2.2057467049999104
inference time for step 6: 2.2218679060001705
inference time for step 7: 2.2317726850001236
inference time for step 8: 2.2561078429998815
inference time for step 9: 2.269036593000237
inference time for step 10: 2.27048238999987
inference time for step 11: 2.2644082399997387
inference time for step 12: 2.2526755089998005
inference time for step 13: 2.2336319870000807
inference time for step 14: 2.2326886160003596
inference time for step 15: 2.230411667000226
inference time for step 16: 2.218204002999755
inference time for step 17: 2.2104489140001533
inference time for step 18: 2.2044161819999317
in

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

a sure sign with him of mental agitation.	این نشان قاطعی از هیجان وی بود شروع به قدم زدن در اتاق کرد.	نشانه ای از اضطراب ذهنی او.	2.4906123264252495
;Cesare, you are worried about this affair,; she said at last.	عاقبت گفت: سزار! تو از این موضوع نگران شده‌ای؛	:کریر، تو نگران این قضیه هستی. سرانجام گفت:	9.425159511373677
;I am very sorry you feel so despondent over it;	بسیار متأسفم که نسبت به آن احساس بی اعتمادی می‌کنی؛	متاسفم که احساس غم و اندوه می کنی.	5.0735520042259505
but I could decide only as seemed right to me.;	اما من فقط بدان علت نتوانستم تصمیم بگیرم که آن کار به نظرم صحیح رسید.	اما من فقط می توانم تصمیم بگیرم.	9.95265213836697
;It is not the affair,; he answered, sullenly; ;I know nothing about it,	مارتینی، با اخم، پاسخ داد: به خاطر آن موضوع نیست، من در آن باره چیزی نمی‌دانم؛	:این قضیه نیست، و با خشونت پاسخ داد: من چیزی	10.27103870297191
and it probably is all right, once you have consented to go into it.	هنگامی که تو قبول می‌کنی در کاری شرکت جویی قطعا کاملا درست است.	و احتمال

In [ ]:
output_file_name = "translation_en-fa_combined_mizan_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### qqp subset

In [ ]:
qqp_original_test, qqp_translated_test = mt_model.load_dataset_test_file(dataset_name="combined", dataset_file="translation_combined_en_fa/test.tsv", source='qqp_train_en_fa')
print(qqp_original_test[0])
print(qqp_translated_test[0])

test part:
 #original: 1104, #translation: 1104
How can I slowly lose weight?
چگونه می توانم کم کم وزن خود را کاهش دهم؟


In [ ]:
!nvidia-smi
!lscpu

Sat Jul  3 20:30:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    33W /  70W |   5486MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(qqp_original_test, qqp_translated_test, device, max_length=512, batch_size=128)

#original_text:1104, #translated_text:1104
#batch: 9
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 2.203811846000008
inference time for step 1: 2.1809551940000347
inference time for step 2: 2.1867584149999857
inference time for step 3: 2.1774539070001993
inference time for step 4: 2.188646286999756
inference time for step 5: 2.190833585999826
inference time for step 6: 2.199965730999793
inference time for step 7: 2.2009575629999745
inference time for step 8: 1.4794907539999258
total inference time: 19.008873282999502
total inference time / #samples: 0.01721818232155752
BLEU Score: 16.473098238322912


In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

How can I slowly lose weight?	چگونه می توانم کم کم وزن خود را کاهش دهم؟	چگونه می توانم کم کم وزنم را از دست بدهم؟	41.11336169005198
How does masturbation affect one's concentrating power?	چگونه استمناء بر قدرت تمرکز شخص تأثیر می گذارد؟	چگونه استمناء بر قدرت تمرکز آمیز انسان تاثیر می گذارد	41.11336169005198
What does it mean when I dreamed of someone and said they're dying?	وقتی خواب کسی را دیدم و گفتم آنها در حال مرگ هستند ، چه معنایی دارد؟	منظور از اینکه من خواب کسی را دیدم و گفتم دارند می میرند	30.215132342213106
How can I modify apk files?	چگونه می توانم فایل های apk را تغییر دهم؟	چگونه می توانم پرونده های APK را تغییر دهم؟	31.02016197007
Where can I buy first copy shoes in mumbai?	از کجا می توانم کفش های کپی درجه اول را در بمبئی بخرم؟	کجا می توانم کفش های اول را در mumbai بخرم؟	40.084360380296616
What is the step by step procedure to apply for Australian PR?	روش مرحله به مرحله برای درخواست شهروندی دایم استرالیا چیست؟	روند قدم به قدم برای استفاده از شرکت تبلیغات استرالیایی	4.9900497

In [ ]:
output_file_name = "translation_en-fa_combined_qqp_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Quran subset

In [15]:
quran_original_test, quran_translated_test = mt_model.load_dataset_test_file(dataset_name="combined", dataset_file="translation_combined_en_fa/test.tsv", source='quran_en_fa')
print(quran_original_test[0])
print(quran_translated_test[0])

test part:
 #original: 5999, #translation: 5999
In the name of Allah, Most Gracious, Most Merciful.
به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خداى بخشاینده مهربان‌///به نام خداوند هستى بخش مهربان///به نام خداوند رحمتگر مهربان///به نام خداوند بخشنده‌ى مهربان.///به نام خداوند بخشنده مهربان///به نام خداوند بخشنده مهربان‌///به نام خداوند بخشنده بخشایشگر///بنام خداوند بخشاینده مهربان‌///به نام خدای رحمتگر بر آفریدگان، رحمتگر بر ویژگان‌


In [ ]:
!nvidia-smi
!lscpu

Sat Jul  3 20:30:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    45W /  70W |   5490MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(quran_original_test, quran_translated_test, device, max_length=512, batch_size=128)

#original_text:5999, #translated_text:5999
#batch: 47
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 2.216846713999985
inference time for step 1: 2.193961683999987
inference time for step 2: 2.206148588999895
inference time for step 3: 2.192960800000037
inference time for step 4: 2.1961606139998366
inference time for step 5: 2.208238722999795
inference time for step 6: 2.193686736000018
inference time for step 7: 2.207011278000209
inference time for step 8: 2.206674735999968
inference time for step 9: 2.209038730999964
inference time for step 10: 2.207963490000111
inference time for step 11: 2.2180355069999678
inference time for step 12: 2.2177766289996725
inference time for step 13: 2.2166543260000253
inference time for step 14: 2.2146837300001607
inference time for step 15: 2.22275072299999
inference time for step 16: 2.2307766869998886
inference time for step 17: 2.222178098999848
inference time for step 18: 2.223492780000015
inference t

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the name of Allah, Most Gracious, Most Merciful.	به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خداى بخشاینده مهربان‌///به نام خداوند هستى بخش مهربان///به نام خداوند رحمتگر مهربان///به نام خداوند بخشنده‌ى مهربان.///به نام خداوند بخشنده مهربان///به نام خداوند بخشنده مهربان‌///به نام خداوند بخشنده بخشایشگر///بنام خداوند بخشاینده مهربان‌///به نام خدای رحمتگر بر آفریدگان، رحمتگر بر ویژگان‌	به نام الله، گرامی ترین و ممنون ترین.	0.0003688819525813449
Praise be to Allah, the Cherisher and Sustainer of the worlds;	همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///ستایش خدا را که پروردگار جهانیان است.///ستایش خداى را که پروردگار جهانیان است///ستایش خدایى را که پروردگار جهانیان،///سپاس و ستایش مخصوص خداوندى است که پروردگار جهانیان است.///ستایش خدای را که پروردگار جهانیان است.///سپاس خداوند را که پروردگار جهانیان است‌///ستایش مخصوص خداوندی است که پروردگار جهانیان است.//

In [ ]:
output_file_name = "translation_en-fa_combined_quran_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

In [13]:
!nvidia-smi
!lscpu

Wed Jul  7 05:25:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    33W /  70W |   6990MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
evaluation_output = mt_model.mt5_evaluation(quran_original_test, quran_translated_test, device, max_length=512, split_reference='///', batch_size=128)

#original_text:5999, #translated_text:5999
#batch: 47
#maximum_translation_reference: 11
Start to evaluate test data ...
inference time for step 0: 2.207366820999823
inference time for step 1: 2.209425366000005
inference time for step 2: 2.225623885999994
inference time for step 3: 2.2580057799998485
inference time for step 4: 2.270689579999953
inference time for step 5: 2.2886298949999855
inference time for step 6: 2.325212884000166
inference time for step 7: 2.3317100849999406
inference time for step 8: 2.3515385369998967
inference time for step 9: 2.3726867780001157
inference time for step 10: 2.4102847180001845
inference time for step 11: 2.4126965640000435
inference time for step 12: 2.409208800999977
inference time for step 13: 2.3758467850000216
inference time for step 14: 2.3319509919999746
inference time for step 15: 2.313040912000133
inference time for step 16: 2.3164967270001853
inference time for step 17: 2.2960531010000977
inference time for step 18: 2.281453575000114
infe

In [17]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the name of Allah, Most Gracious, Most Merciful.	به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خداى بخشاینده مهربان‌///به نام خداوند هستى بخش مهربان///به نام خداوند رحمتگر مهربان///به نام خداوند بخشنده‌ى مهربان.///به نام خداوند بخشنده مهربان///به نام خداوند بخشنده مهربان‌///به نام خداوند بخشنده بخشایشگر///بنام خداوند بخشاینده مهربان‌///به نام خدای رحمتگر بر آفریدگان، رحمتگر بر ویژگان‌	به نام الله، گرامی ترین و ممنون ترین.	11.339582221952005
Praise be to Allah, the Cherisher and Sustainer of the worlds;	همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///ستایش خدا را که پروردگار جهانیان است.///ستایش خداى را که پروردگار جهانیان است///ستایش خدایى را که پروردگار جهانیان،///سپاس و ستایش مخصوص خداوندى است که پروردگار جهانیان است.///ستایش خدای را که پروردگار جهانیان است.///سپاس خداوند را که پروردگار جهانیان است‌///ستایش مخصوص خداوندی است که پروردگار جهانیان است.///سپ

In [18]:
output_file_name = "translation_en-fa_combined_quran_split-refs_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

## QQP Dataset

## TEP Dataset


## OPUS Dataset

## Global Voice Dataset